<a href="https://colab.research.google.com/github/nastaran-farhadi/BBAC-with-LLM-in-healthcare/blob/main/bbac_multi_user_hospital.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
!pip install faker pandas
from faker import Faker
import pandas as pd
import random
from datetime import datetime, timedelta
fake = Faker()

In [13]:
import os
BASE_DIR = '/content/drive/MyDrive/bbac_multiuser'

Step 1: the data set that employees want to get access

In [14]:
STORAGE_DIR = os.path.join(BASE_DIR, 'Storage')
os.makedirs(STORAGE_DIR, exist_ok=True)

ROLE_DEPARTMENT_MAP = {
    "Doctor": ["Emergency", "ICU", "Radiology", "Surgery", "Pediatrics"],
    "Nurse": ["Emergency", "ICU", "Pediatrics", "Surgery"],
    "Receptionist": ["Emergency", "Records", "Admissions"],
    "Technician": ["Radiology", "Laboratory", "Pharmacy"],
    "Admin": ["Administration", "HR", "Records"]
}

SENSITIVE_ILLNESSES = ["HIV", "Mental Health Disorder", "Substance Abuse", "Cancer"]
REGULAR_ILLNESSES = ["Flu", "COVID-19", "Broken Bone", "Infection", "Hypertension", "Diabetes"]

def generate_employee_dataset(n):
    """Generate enhanced employee dataset with access permissions"""
    data = []
    for _ in range(n):
        role = random.choice(list(ROLE_DEPARTMENT_MAP.keys()))
        department = random.choice(ROLE_DEPARTMENT_MAP[role])

        # Enhanced permissions structure
        permissions = {
            "patient_data": {
                "access_level": "Full" if role == "Doctor" else "Limited",
                "allowed_operations": ["Read", "Update"] if role == "Doctor" else ["Read"],
                "sensitivity_access": ["High"] if role in ["Doctor", "Senior Nurse"] else ["Normal"]
            },
            "employee_data": {
                "access_level": "Admin" if role == "Admin" else "None",
                "allowed_operations": ["Read", "Write", "Update"] if role == "Admin" else []
            }
        }

        data.append({
            "Employee_ID": f"E{random.randint(1, 999):03d}",
            "Employee_Name": fake.name(),
            "Role": role,
            "Department": department,
            "Salary": random.randint(50000, 200000),
            "Address": fake.address().replace("\n", ", "),
            "Contact": fake.phone_number(),
            "Access_Permissions": permissions,
            "Hire_Date": datetime.now() - timedelta(days=random.randint(365, 365*5))
        })

    return pd.DataFrame(data)

def generate_patient_dataset(n):
    """Generate patient data with enhanced medical records"""
    data = []
    for _ in range(n):
        is_sensitive = random.random() < 0.2
        illness = random.choice(SENSITIVE_ILLNESSES if is_sensitive else REGULAR_ILLNESSES)

        data.append({
            "Patient_ID": f"P{random.randint(1, 999):03d}",
            "Patient_Name": fake.name(),
            "Gender": random.choice(["Male", "Female", "Other"]),
            "Age": random.randint(1, 100),
            "Address": fake.address().replace("\n", ", "),
            "Emergency_Contact": fake.phone_number(),
            "Primary_Diagnosis": illness,
            "Insurance_Type": random.choice(["Private", "Medicare", "Medicaid", "Self-Pay"]),
            "Admission_Date": datetime.now() - timedelta(days=random.randint(0, 30)),
            "Room_Number": random.randint(100, 500),
            "Data_Sensitivity": "High" if is_sensitive else "Normal",
            "Attending_Physician": f"E{random.randint(1, 999):03d}",
            "Treatment_Plan": random.choice(["Inpatient", "Outpatient"])
        })

    return pd.DataFrame(data)

# ======================
# DATA GENERATION & STORAGE
# ======================

# Generate datasets
employee_df = generate_employee_dataset(50)  # 50 employees
patient_df = generate_patient_dataset(200)   # 200 patients

# Save to Storage directory
employee_path = os.path.join(STORAGE_DIR, 'employee_records.csv')
patient_path = os.path.join(STORAGE_DIR, 'patient_records.csv')

employee_df.to_csv(employee_path, index=False)
patient_df.to_csv(patient_path, index=False)

print(f"Employee records saved to: {employee_path}")
print(f"Patient records saved to: {patient_path}")

# Optional: Add historical data generation from previous implementation
# (Include your historical data generation code here)

Employee records saved to: /content/drive/MyDrive/bbac_multiuser/Storage/employee_records.csv
Patient records saved to: /content/drive/MyDrive/bbac_multiuser/Storage/patient_records.csv


Step 2:Creating historical dataset of several users in 10 days

In [15]:
HISTORICAL_DIR = os.path.join(BASE_DIR, 'Historical_data')
os.makedirs(HISTORICAL_DIR, exist_ok=True)
# ======================
# CONFIGURATION
# ======================
NUM_USERS = 10
BASE_DATE = datetime(2023, 10, 1)

ANOMALY_PROBABILITY = 0.3



# ======================
# CORE FUNCTIONS
# ======================
# (Keep all the functions from previous code exactly the same)
# [generate_user_profile, calculate_behavior_score,
#  generate_normal_behavior, inject_anomalies]
def generate_user_profile(user_id):
    """Generate static user profile with access permissions"""
    return {
        'employee_id': f'E-{2023 + user_id}',
        'name': fake.name(),
        'role': random.choice(['Doctor', 'Nurse', 'Technician']),
        'department': random.choice(['Cardiology', 'Neurology', 'Oncology']),
        'access_permissions': {
            'allowed_datasets': ['Patient-records','employee-records'],
            'allowed_operations': ['Read', 'Update'],
            'data_categories': ['Medical History', 'Treatment Plans']
        }
    }

def calculate_behavior_score(access_time, ip_address, mfa_failures=0):
    """Calculate risk score based on multiple factors"""
    score = 0.0

    # Time-based scoring (8AM is ideal)
    hour = access_time.hour
    if 7 <= hour <= 9:
        score += 0.0
    else:
        score += min(abs(hour - 8) * 0.1, 0.5)

    # Network location scoring
    if "Internal" in ip_address:
        score += 0.0
    else:
        score += 0.4  # Increased weight for external access

    # Authentication failures
    score += min(mfa_failures * 0.15, 0.45)

    # Random variation
    score += random.uniform(-0.05, 0.05)

    return max(0.0, min(round(score, 2), 1.0))

def generate_normal_behavior(base_date, user_profile):
    """Generate 10 days of normal access patterns"""
    base_ip = '10.1.1.15 (Internal)'
    base_location = 'Main Campus - ' + user_profile['department']
    normal_access_time = datetime.strptime("08:00", "%H:%M").time()

    records = []
    for day in range(10):
        access_date = base_date + timedelta(days=day)
        time_variation = timedelta(minutes=random.randint(-15, 15))
        access_time = datetime.combine(access_date, normal_access_time) + time_variation

        record = {
            'date': access_date.strftime("%Y-%m-%d"),
            'timestamp': access_time.strftime("%Y-%m-%d %H:%M:%S"),
            'requested_dataset': 'Patient Records',
            'requested_operation': random.choices(['Read', 'Update'], weights=[0.85, 0.15])[0],
            'access_outcome': 'Granted',
            'mfa_failures': random.choices([0, 1], weights=[0.95, 0.05])[0],
            'ip_address': base_ip,
            'location': base_location,
            'behavior_score': calculate_behavior_score(access_time, base_ip)
        }

        # Add minor time jitter
        if random.random() < 0.1:
            record['timestamp'] = (access_time + timedelta(minutes=random.randint(-30, 30))).strftime("%Y-%m-%d %H:%M:%S")

        records.append(record)

    return records

def inject_anomalies(records, user_profile):
    """Inject realistic anomalous patterns into the dataset"""
    anomalous_records = []

    # 1. Night Access Pattern
    if random.random() < ANOMALY_PROBABILITY:
        anomaly_date = datetime.strptime(records[-1]['date'], "%Y-%m-%d") + timedelta(days=1)
        anomaly_time = datetime.strptime("02:00", "%H:%M").time()
        anomalous_records.append({
            'date': anomaly_date.strftime("%Y-%m-%d"),
            'timestamp': datetime.combine(anomaly_date, anomaly_time).strftime("%Y-%m-%d %H:%M:%S"),
            'requested_dataset': 'Patient Records',
            'requested_operation': 'Update',
            'access_outcome': 'Denied',
            'mfa_failures': random.randint(2, 3),
            'ip_address': '192.168.1.100 (External)',
            'location': 'Unknown Location',
            'behavior_score': 0.85
        })

    # 2. Unauthorized Dataset Access
    if random.random() < ANOMALY_PROBABILITY/2:
        original_record = random.choice(records).copy()
        anomalous_records.append({
            **original_record,
            'requested_dataset': 'Clinical Trials',
            'access_outcome': 'Denied',
            'behavior_score': min(original_record['behavior_score'] + 0.4, 1.0)
        })

    # 3. Rapid Fire Access Attempts
    if random.random() < ANOMALY_PROBABILITY/3:
        base_record = random.choice(records).copy()
        for i in range(3):
            anomalous_records.append({
                **base_record,
                'timestamp': (datetime.strptime(base_record['timestamp'], "%Y-%m-%d %H:%M:%S") +
                            timedelta(minutes=i*2)).strftime("%Y-%m-%d %H:%M:%S"),
                'mfa_failures': 1,
                'behavior_score': min(base_record['behavior_score'] + 0.25*(i+1), 1.0)
            })

    return records + anomalous_records

# ======================
# DATA GENERATION & SAVING
# ======================
for user_id in range(1, NUM_USERS + 1):
    user_profile = generate_user_profile(user_id)
    normal_data = generate_normal_behavior(BASE_DATE, user_profile)
    full_data = inject_anomalies(normal_data, user_profile)
    historical_df = pd.DataFrame(full_data)

    # Create filename with full path
    filename = os.path.join(HISTORICAL_DIR, f"{user_profile['employee_id']}_historical_data.txt")

    # Write to file
    with open(filename, "w") as f:
        # File header
        f.write(f"=== User Profile: {user_profile['name']} ===\n")
        f.write(f"Employee ID: {user_profile['employee_id']}\n")
        f.write(f"Department: {user_profile['department']}\n")
        f.write(f"Role: {user_profile['role']}\n\n")

        f.write("=== Access Log ===\n")
        historical_df.to_csv(f, index=False, sep='\t')  # Using tab separation for better readability

    print(f"Saved {user_profile['employee_id']}'s data to Google Drive")

print("\nOperation completed. Files saved to:")
print(f"Google Drive Path: {HISTORICAL_DIR}")

# Save the historical data as csv
CSV_DIR = os.path.join(BASE_DIR, 'CSV_Data')
os.makedirs(CSV_DIR, exist_ok=True)

for user_id in range(1, NUM_USERS + 1):
    user_profile = generate_user_profile(user_id)
    normal_data = generate_normal_behavior(BASE_DATE, user_profile)
    full_data = inject_anomalies(normal_data, user_profile)

    # Create a DataFrame for historical data
    historical_df = pd.DataFrame(full_data)

    # Add user profile columns to historical data
    historical_df['User Profile'] = user_profile['name']
    historical_df['Employee ID'] = user_profile['employee_id']
    historical_df['Department'] = user_profile['department']
    historical_df['Role'] = user_profile['role']

    # Reorder columns
    desired_order = ['User Profile', 'Employee ID', 'Department', 'Role',
                     'timestamp', 'requested_dataset', 'requested_operation',
                     'access_outcome', 'mfa_failures', 'ip_address', 'location',
                     'behavior_score']
    historical_df = historical_df[desired_order]  # Apply the desired order

    # Save CSV file
    csv_filename = os.path.join(CSV_DIR, f"{user_profile['employee_id']}_historical_data.csv")
    historical_df.to_csv(csv_filename, index=False)
    print(f"Saved {user_profile['employee_id']}'s data to {csv_filename}")

print("\nOperation completed. Files saved to:")
print(f"CSV Data Path: {CSV_DIR}")


Saved E-2024's data to Google Drive
Saved E-2025's data to Google Drive
Saved E-2026's data to Google Drive
Saved E-2027's data to Google Drive
Saved E-2028's data to Google Drive
Saved E-2029's data to Google Drive
Saved E-2030's data to Google Drive
Saved E-2031's data to Google Drive
Saved E-2032's data to Google Drive
Saved E-2033's data to Google Drive

Operation completed. Files saved to:
Google Drive Path: /content/drive/MyDrive/bbac_multiuser/Historical_data
Saved E-2024's data to /content/drive/MyDrive/bbac_multiuser/CSV_Data/E-2024_historical_data.csv
Saved E-2025's data to /content/drive/MyDrive/bbac_multiuser/CSV_Data/E-2025_historical_data.csv
Saved E-2026's data to /content/drive/MyDrive/bbac_multiuser/CSV_Data/E-2026_historical_data.csv
Saved E-2027's data to /content/drive/MyDrive/bbac_multiuser/CSV_Data/E-2027_historical_data.csv
Saved E-2028's data to /content/drive/MyDrive/bbac_multiuser/CSV_Data/E-2028_historical_data.csv
Saved E-2029's data to /content/drive/MyDriv

Step3: key pattern of historical dataset of several users

In [9]:
!pip install faiss-cpu # Installing faiss-cpu, appropriate for CPU-based systems. If you have a GPU, you can install faiss-gpu instead.
from transformers import AutoTokenizer, AutoModel
import faiss
import json
from sentence_transformers import SentenceTransformer

In [16]:

# Configuration
Embeddings_dir = os.path.join(BASE_DIR, 'Embeddings')
os.makedirs(Embeddings_dir, exist_ok=True)

# Load the SentenceTransformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

def embed_and_save(csv_file):
    """Embeds the data in the given CSV file and saves it with embeddings."""
    try:
        # Load the CSV data
        df = pd.read_csv(csv_file)

        # Create text representations for embedding
        df['log_text'] = df.apply(
            lambda x: f"On {x['timestamp'].split()[0]} at {x['timestamp'].split()[1]}, " +  # Assuming timestamp is in 'YYYY-MM-DD HH:MM:SS' format
                      f"{x['User Profile']} performed {x['requested_operation']} " +
                      f"on {x['requested_dataset']} from {x['location']} (IP: {x['ip_address']}), " +
                      f"MFA failures: {x['mfa_failures']}, " +
                      f"Risk score: {x['behavior_score']}",
            axis=1
        )

        # Generate embeddings
        embeddings = model.encode(df['log_text'].tolist(), show_progress_bar=True)
        df['embedding'] = list(embeddings)  # Add embeddings as a new column

        # Save the DataFrame with embeddings
        output_file = os.path.join(Embeddings_dir, os.path.basename(csv_file))  # Same filename, different directory
        df.to_csv(output_file, index=False)
        print(f"Saved embeddings to: {output_file}")

    except Exception as e:
        print(f"Error processing {csv_file}: {str(e)}")


# Process all CSV files
csv_files = [f for f in os.listdir(CSV_DIR) if f.endswith('_historical_data.csv')]
for csv_file in csv_files:
    file_path = os.path.join(CSV_DIR, csv_file)
    embed_and_save(file_path)

print("Embedding process completed.")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Saved embeddings to: /content/drive/MyDrive/bbac_multiuser/Embeddings/E-2024_historical_data.csv


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Saved embeddings to: /content/drive/MyDrive/bbac_multiuser/Embeddings/E-2025_historical_data.csv


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Saved embeddings to: /content/drive/MyDrive/bbac_multiuser/Embeddings/E-2031_historical_data.csv


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Saved embeddings to: /content/drive/MyDrive/bbac_multiuser/Embeddings/E-2032_historical_data.csv


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Saved embeddings to: /content/drive/MyDrive/bbac_multiuser/Embeddings/E-2033_historical_data.csv
Embedding process completed.


In [20]:
import faiss
import numpy as np
!pip install -U langchain-community
from langchain.vectorstores import FAISS
from langchain.docstore.in_memory import InMemoryDocstore
from langchain.schema import Document

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00


In [21]:

# --- Vector Database with Hugging Face Embeddings ---
def create_vector_db_hf(csv_files_dir):
    """Store Hugging Face embeddings from multiple CSV files in a searchable vector database"""

    all_texts = []
    all_embeddings = []

    for filename in os.listdir(csv_files_dir):
        if filename.endswith(".csv"):
            filepath = os.path.join(csv_files_dir, filename)
            embedded_df = pd.read_csv(filepath)

            all_texts.extend(embedded_df['log_text'].tolist())
            all_embeddings.extend(embedded_df['embedding'].apply(lambda x: np.fromstring(x[1:-1], sep=',').astype('float32')).tolist())

    # Convert to numpy arrays for FAISS
    all_embeddings = np.array(all_embeddings)

    # Create FAISS index
    dimension = all_embeddings.shape[1]  # Get embedding dimension
    index = faiss.IndexFlatL2(dimension)  # Create FAISS index
    index.add(all_embeddings)  # Add embeddings

    # Create LangChain FAISS wrapper
    vector_db = FAISS(
        embedding_function=None,  # Not needed since we have precomputed embeddings
        index=index,
        docstore=InMemoryDocstore(),
        index_to_docstore_id={i: i for i in range(len(all_texts))}
    )

    # Add text metadata
    for i, text in enumerate(all_texts):
        vector_db.docstore.add({i: Document(page_content=text)})

    return vector_db



# Create and persist vector DB
vector_db = create_vector_db_hf(Embeddings_dir)

print("\n=== Vector Database Info ===")
print(f"Index size: {vector_db.index.ntotal}")

<ipython-input-21-d69abc535490>:14: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  all_embeddings.extend(embedded_df['embedding'].apply(lambda x: np.fromstring(x[1:-1], sep=',').astype('float32')).tolist())
<ipython-input-21-d69abc535490>:14: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  all_embeddings.extend(embedded_df['embedding'].apply(lambda x: np.fromstring(x[1:-1], sep=',').astype('float32')).tolist())
<ipython-input-21-d69abc535490>:14: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  all_embeddings.extend(embedded_df['embedding'].apply(lambda x: np.fromstring(x[1:-1], sep=',').astype('float32')).tolist())
<ipython-input-21-d69abc535490>:14: DeprecationWarning: string or file could not be read to its end due to unmatched data


=== Vector Database Info ===
Index size: 103


Step 4: realtime user choosen from historical datasetrequest that could be normal behavior or suspious behavior

In [23]:
def get_random_user_profile(csv_data_dir):
    """
    Randomly selects a user profile from the historical dataset in the CSV_Data folder.

    Args:
        csv_data_dir (str): Path to the directory containing historical data CSV files.

    Returns:
        dict: A randomly selected user profile.
    """
    # Load all historical data files
    historical_files = [f for f in os.listdir(csv_data_dir) if f.endswith('.csv')]

    if not historical_files:
        raise FileNotFoundError(f"No CSV files found in {csv_data_dir}")

    # Load all historical data into a single DataFrame
    historical_data = pd.concat(
        [pd.read_csv(os.path.join(csv_data_dir, f)) for f in historical_files],
        ignore_index=True
    )

    # Randomly select a user
    random_user = historical_data.sample(n=1).iloc[0]

    # Create user profile
    user_profile = {
        'employee_id': random_user['Employee ID'],
        'name': random_user['User Profile'],
        'role': random_user['Role'],
        'department': random_user['Department']
    }

    return user_profile

def simulate_real_time_request_from_profile(user_profile, csv_data_dir, is_suspicious=False):
    """
    Simulates a real-time access request by combining static attributes from the user's profile
    with dynamic attributes. If suspicious, it changes 2-3 attributes based on historical data.

    Args:
        user_profile (dict): Static user info (e.g., name, role, department, employee_id)
        csv_data_dir (str): Path to the directory containing historical data CSV files.
        is_suspicious (bool): Flag to indicate if the dynamic attributes should simulate malicious behavior

    Returns:
        dict: Combined request with both static and dynamic fields.
    """
    # Static attributes
    request = {
        'employee_id': user_profile.get('employee_id'),
        'name': user_profile.get('name'),
        'role': user_profile.get('role'),
        'department': user_profile.get('department'),
        'dataset': 'Patient Records'  # Assuming the dataset is fixed for now
    }

    # Dynamic attributes (default values for normal behavior)
    dynamic_attributes = {
        'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'requested_operation': random.choice(['Read', 'Update']),
        'access_outcome': 'Granted',
        'mfa_failures': random.choices([0, 1], weights=[0.95, 0.05])[0],
        'ip_address': '10.1.1.15 (Internal)',
        'location': f"Main Campus - {user_profile.get('department')}",
        'behavior_score': round(random.uniform(0.0, 0.3), 2)  # Normal behavior score
    }

    if is_suspicious:
        # Load all historical data files
        historical_files = [f for f in os.listdir(csv_data_dir) if f.endswith('.csv')]

        if not historical_files:
            print(f"No CSV files found in {csv_data_dir}")
        else:
            # Load all historical data into a single DataFrame
            historical_data = pd.concat(
                [pd.read_csv(os.path.join(csv_data_dir, f)) for f in historical_files],
                ignore_index=True
            )

            # Filter for the specific user's data
            user_data = historical_data[historical_data['Employee ID'] == user_profile.get('employee_id')]

            if not user_data.empty:
                # Filter for suspicious behavior (high behavior_score)
                suspicious_data = user_data[user_data['behavior_score'] >= 0.5]

                if not suspicious_data.empty:
                    # Randomly select a row for suspicious behavior inspiration
                    suspicious_row = suspicious_data.sample(n=1).iloc[0]

                    # Randomly change 2-3 attributes
                    attributes_to_change = random.sample([
                        'timestamp', 'requested_operation', 'access_outcome',
                        'mfa_failures', 'ip_address', 'location'
                    ], k=random.randint(2, 3))

                    for attr in attributes_to_change:
                        dynamic_attributes[attr] = suspicious_row[attr]

                    # Increase behavior score for suspicious requests
                    dynamic_attributes['behavior_score'] = round(random.uniform(0.7, 1.0), 2)
                else:
                    print(f"No suspicious historical data found for {user_profile.get('employee_id')}")
            else:
                print(f"No historical data found for {user_profile.get('employee_id')}")

    # Combine static and dynamic attributes
    request.update(dynamic_attributes)
    return request

# Example Usage
if __name__ == "__main__":
    # Get a random user profile
    try:
        user_profile = get_random_user_profile(CSV_DIR)
        print(f"Selected User: {user_profile}")

        # Simulate a normal request
        normal_request = simulate_real_time_request_from_profile(user_profile, CSV_DIR)
        print("\n=== Normal Request ===")
        print(normal_request)

        # Simulate a suspicious request
        suspicious_request = simulate_real_time_request_from_profile(user_profile, CSV_DIR, is_suspicious=True)
        print("\n=== Suspicious Request ===")
        print(suspicious_request)
    except Exception as e:
        print(f"Error: {str(e)}")

Selected User: {'employee_id': 'E-2031', 'name': 'Christopher Hutchinson', 'role': 'Nurse', 'department': 'Neurology'}

=== Normal Request ===
{'employee_id': 'E-2031', 'name': 'Christopher Hutchinson', 'role': 'Nurse', 'department': 'Neurology', 'dataset': 'Patient Records', 'timestamp': '2025-03-25 11:48:27', 'requested_operation': 'Read', 'access_outcome': 'Granted', 'mfa_failures': 0, 'ip_address': '10.1.1.15 (Internal)', 'location': 'Main Campus - Neurology', 'behavior_score': 0.28}
No suspicious historical data found for E-2031

=== Suspicious Request ===
{'employee_id': 'E-2031', 'name': 'Christopher Hutchinson', 'role': 'Nurse', 'department': 'Neurology', 'dataset': 'Patient Records', 'timestamp': '2025-03-25 11:48:27', 'requested_operation': 'Read', 'access_outcome': 'Granted', 'mfa_failures': 0, 'ip_address': '10.1.1.15 (Internal)', 'location': 'Main Campus - Neurology', 'behavior_score': 0.26}


In [24]:
# ... (Import necessary libraries and define get_random_user_profile and simulate_real_time_request_from_profile functions if they're not already defined) ...

# --- Step 2: Convert the request to a textual query ---
def request_to_text(request):
    """Converts a request dictionary into a textual representation for similarity search."""
    return (f"On {request['timestamp'].split()[0]} at {request['timestamp'].split()[1]}, "  # Assuming timestamp is in 'YYYY-MM-DD HH:MM:SS' format
            f"{request['name']} ({request['role']} in {request['department']}) performed {request['requested_operation']} "
            f"on {request['dataset']} from {request['location']} (IP: {request['ip_address']}), "
            f"MFA failures: {request['mfa_failures']}, "
            f"Risk score: {request['behavior_score']}")


# Assuming you have normal_request and suspicious_request from the previous step
# If not, you'll need to define them or load them from where they were stored

# Convert normal_request to text
normal_request_text = request_to_text(normal_request)
print("\n=== Normal Request Text ===")
print(normal_request_text)

# Convert suspicious_request to text (if needed)
suspicious_request_text = request_to_text(suspicious_request)
print("\n=== Suspicious Request Text ===")
print(suspicious_request_text)


=== Normal Request Text ===
On 2025-03-25 at 11:48:27, Christopher Hutchinson (Nurse in Neurology) performed Read on Patient Records from Main Campus - Neurology (IP: 10.1.1.15 (Internal)), MFA failures: 0, Risk score: 0.28

=== Suspicious Request Text ===
On 2025-03-25 at 11:48:27, Christopher Hutchinson (Nurse in Neurology) performed Read on Patient Records from Main Campus - Neurology (IP: 10.1.1.15 (Internal)), MFA failures: 0, Risk score: 0.26


Step 5: similirity search

In [25]:
# --- Initialize the embedding model ---
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# --- Generate embeddings for your documents (example) ---
documents = ["log entry 1", "log entry 2", ...]  # Replace with your actual log entries
document_embeddings = embedding_model.encode(documents)  # Shape: (num_docs, 384)

# --- Normalize embeddings for cosine similarity ---
faiss.normalize_L2(document_embeddings)

# --- Create a FAISS index with dimension 384 ---
index = faiss.IndexFlatIP(384)  # Inner product (cosine similarity) index
index.add(document_embeddings.astype('float32'))  # Add normalized embeddings

# --- Rebuild your vector_db with the new index and docstore ---
# (Assuming `vector_db` is a custom class with `index` and `docstore`)
vector_db.index = index
vector_db.docstore = {i: doc for i, doc in enumerate(documents)}  # Replace with your docstore logic

In [26]:


# --- Step 2: Encode the query using the same sentence transformer model ---
query_text = request_to_text(normal_request)
query_embedding = embedding_model.encode(query_text)

# --- Step 3: Reshape to 2D and normalize ---
# --- When querying ---
query_embedding = embedding_model.encode(query_text).reshape(1, -1)
faiss.normalize_L2(query_embedding)  # Normalize query embedding

# --- Step 4: Retrieve similar historical entries from FAISS vector DB using cosine similarity---
k = 4
distances, indices = vector_db.index.search(query_embedding.astype('float32'), k=k)
# --- Debug: Check FAISS Output ---
print("Indices:", indices)
print("Distances:", distances)
# --- Step 5: Retrieve logs (FIXED) ---
retrieved_logs = []
for idx, similarity in zip(indices[0], distances[0]):  # Use distances directly
    if 0 <= idx < len(vector_db.docstore):
        doc = vector_db.docstore.get(idx)
        if doc is not None:
            retrieved_logs.append({
                'log': doc,
                'similarity': similarity  # Directly use the inner product
            })

# --- Step 6: Display logs ---
for log_data in retrieved_logs:
    print(f"Log: {log_data['log']}")
    print(f"Similarity: {log_data['similarity']:.4f}\n")

Indices: [[ 1  0  2 -1]]
Distances: [[ 1.3817665e-01  1.3684626e-01 -9.4397701e-03 -3.4028235e+38]]
Log: log entry 2
Similarity: 0.1382

Log: log entry 1
Similarity: 0.1368

Log: Ellipsis
Similarity: -0.0094



Step 6: Apply RAG

In [27]:
!pip install faiss-cpu sentence-transformers langchain-community
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
import re

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [30]:
def analyze_and_decide_with_llm(similar_logs, request, threshold=0.8):
    """Analyzes retrieved logs and decides using an LLM."""

    # Prepare context for the language model
    context = "\n".join([f"Similarity: {log_data['similarity']:.4f}, Log: {log_data['log']}"
                        for log_data in similar_logs])

    # Create a prompt template
    prompt_template = PromptTemplate(
        input_variables=["request", "context"],
        template="""
        You are an expert security analyst evaluating a real-time user access request in a healthcare system.

        Access Request:
        {request}

        Similar Historical Logs:
        {context}

        Based on the access request and the historical logs provided:
        1. Evaluate whether the access request is consistent with normal behavior.
        2. Consider factors such as the user's role, department, the requested operation, the risk score, and any unusual patterns shown in the similar logs.
        3. Answer with exactly one of the following:
         - "Decision: Yes" (if the access should be granted), or
         - "Decision: No" (if the access should be denied).

        Then, provide a concise justification for your decision in one or two sentences.

        Your answer should be structured as follows:
        Decision: <Yes/No>
        Justification: <your justification>
        """
)

    # Format request as text
    request_str = f"User: {request['name']}, Role: {request['role']}, Department: {request['department']}, " \
                f"Action: {request['requested_operation']}, Dataset: {request['dataset']}, " \
                f"Timestamp: {request['timestamp']}, Location: {request['location']}, " \
                f"IP: {request['ip_address']}, MFA Failures: {request['mfa_failures']}, " \
                f"Behavior Score: {request['behavior_score']}"

    # Generate prompt
    prompt = prompt_template.format(request=request_str, context=context)

    # Initialize the language model with HARDCODED API KEY (INSECURE!)
    llm = ChatOpenAI(
        model_name="gpt-4",
        temperature=0,
        openai_api_key=""  # ⚠️ INSECURE - REPLACE WITH YOUR KEY
    )

    try:
        # Get LLM response
        response = llm.invoke(prompt).content

        # Extract decision and justification
        decision_match = re.search(r"Decision:\s*(Yes|No)", response, re.IGNORECASE)
        justification_match = re.search(r"Justify your decision:(.*)", response, re.DOTALL)

        if decision_match:
            decision = decision_match.group(1).strip().lower()
            grant_access = decision == "yes"
        else:
            print("Error parsing decision")
            grant_access = False

        justification = justification_match.group(1).strip() if justification_match else "No justification"
        return grant_access, justification

    except Exception as e:
        print(f"Error: {e}")
        return False, "LLM processing failed"



In [34]:
# Convert the normal request to text (as shown previously)
normal_request_text = request_to_text(normal_request)
print("\n=== Normal Request Text ===")
print(normal_request_text)

#Display the retrieved logs
print("\n=== Retrieved Logs ===")
for log_data in retrieved_logs:
    print(f"Log: {log_data['log']}")
    print(f"Similarity: {log_data['similarity']:.4f}\n")




# Use LLM with RAG to decide
grant_access, justification = analyze_and_decide_with_llm(retrieved_logs, normal_request)

print("\n=== Decision ===")
if grant_access:
    print("Access Granted.")
else:
    print("Access Denied.")
print("Justification:", justification)



=== Normal Request Text ===
On 2025-03-25 at 11:48:27, Christopher Hutchinson (Nurse in Neurology) performed Read on Patient Records from Main Campus - Neurology (IP: 10.1.1.15 (Internal)), MFA failures: 0, Risk score: 0.28

=== Retrieved Logs ===
Log: log entry 2
Similarity: 0.1382

Log: log entry 1
Similarity: 0.1368

Log: Ellipsis
Similarity: -0.0094


=== Decision ===
Access Granted.
Justification: No justification
